In [1]:
import librosa
import json
import os

In [72]:
with open('answer.json', encoding="UTF-8") as f:
    json_data = json.load(f)

In [73]:
path = "./P1"
file_list = os.listdir(path)

In [76]:
for i, f in enumerate(file_list):
    y, sr = librosa.load(path + '/' + f,sr=None)
    duration = librosa.get_duration(y=y, sr=sr)
    json_data['Q1'][i]['filename'] = f
    json_data['Q1'][i]['duration'] = float(f"{duration:.3f}")

with open('./answer.json', 'w', encoding='UTF-8') as make_file:
    json.dump(json_data, make_file, indent="\t", ensure_ascii=False)

---
1.2. 주어진 wav 파일 내 subchunk "THIS" 섹션의 데이터를 분석하시오(4, 5번)

In [60]:
# -*- coding: utf-8 -*-
"""
Created on Fri Oct  9 15:00:29 2020

@author: r
"""

from ctypes import *


def readFourcc(f):
    return f.read(4).decode()
    
def readBytesAsInt(f,size):
    if size == 0 or size > 4:
        return 0
    
    data = bytes(f.read(size))
    if size == 1:
        return data[0]
    elif size == 2:
        return data[0] | data[1] <<8
    elif size == 3:
        return data[0] | data[1] <<8 | data[2] << 16
    else:
        return data[0] | data[1] <<8 | data[2] << 16 | data[3] << 24
    return 0

def readBytesAsfloat(f,size=4):
    if size != 4:
        return 0
    data = bytes(f.read(size))
    data = data[0] | data[1] <<8 | data[2] << 16 | data[3] << 24
    
    #https://stackoverflow.com/questions/1592158/convert-hex-to-float
    cp = pointer(c_int(data))           # make this into a c integer
    fp = cast(cp, POINTER(c_float))  # cast the int pointer to a float pointer
    return fp.contents.value         # dereference the pointer, get the float


class fmt_chunk():
    wFormatTag = 0
    wChannels = 0
    dwSamplePerSec = 0
    dwAvgBytesPerSec = 0
    wBlockAlign = 0
    wBitsPerSample = 0
    cbsize = 0
    format_specific_data = []
    
    def parse(self, f, cksize):
        
        self.wFormatTag = readBytesAsInt(f,2)
        self.wChannels = readBytesAsInt(f,2)
        self.dwSamplePerSec = readBytesAsInt(f,4)
        self.dwAvgBytesPerSec = readBytesAsInt(f,4)
        self.wBlockAlign = readBytesAsInt(f,2)
        self.wBitsPerSample = readBytesAsInt(f,2)
        
        cksize -= 16
        
        print('  wFormatTag :',self.wFormatTag)
        print('  wChannels :',self.wChannels)
        print('  dwSamplePerSec :',self.dwSamplePerSec)
        print('  dwAvgBytesPerSec :',self.dwAvgBytesPerSec)
        print('  wBlockAlign :',self.wBlockAlign)
        print('  wBitsPerSample :',self.wBitsPerSample)
        
        if cksize > 0:
            self.cbsize = readBytesAsInt(f,2)
            cksize -= 2
            print('  cbsize :',self.cbsize)
            
            if self.cbsize > 0 and cksize >= self.cbsize:
                self.format_specific_data = bytes(f.read(self.cbsize))
                print('  format specific data : 0x',self.format_specific_data.hex())
                cksize -= self.cbsize
        
        if cksize > 0:
            f.read(cksize)
        
        return 0

class peak_chunk():
    version = 0
    timestamp = 0
    value = []
    position = []
    
    
    def parse(self, f,cksize):
        self.version = readBytesAsInt(f,4)
        self.timestamp = readBytesAsInt(f,4)
        cksize -= 8
        
        print('  version :',self.version)
        print('  timestamp :',self.timestamp)
        
        self.value = []
        self.position = []
        
        while cksize > 0:
            value = readBytesAsfloat(f,4)
            position = readBytesAsInt(f,4)
            
            self.value.append(value)
            self.position.append(position)
            
            print('  value :',value)
            print('  position :',position)
            cksize -= 8
            
        return 0
    
class fact_chunk():
    dwSampleLength = 0
    
    def parse(self, f,cksize):
        self.dwSampleLength = readBytesAsInt(f,4)
        print('  dwSampleLength :',self.dwSampleLength)
        if cksize > 4:
            _ = f.read(cksize-4)
        return 0
    


class cue_chunk():
    dwCuePoints = 0
    cues = []
    
    class cue_point():        
        def __init__(self, f):
            self.dwName = readFourcc(f)
            self.dwposition = readBytesAsInt(f,4)
            self.fccChunk = readFourcc(f)
            self.dwChunkStart = readBytesAsInt(f,4) 
            self.dwBlockStart = readBytesAsInt(f,4)
            self.dwSampleOffset = readBytesAsInt(f,4)
            print('  dwName :',self.dwName)
            print('  dwposition :',self.dwposition)
            print('  fccChunk :',self.fccChunk)
            print('  dwChunkStart :',self.dwChunkStart)
            print('  dwBlockStart :',self.dwBlockStart)
            print('  dwSampleOffset :',self.dwSampleOffset)
    
    def parse(self,f,cksize):
        self.dwCuePoints = readBytesAsInt(f,4)
        cksize -= 4
        print('  dwCuePoints :',self.dwCuePoints)
        for _ in range(self.dwCuePoints):
            cue = self.cue_point(f)
            cksize -= 24
            self.cues.append(cue)
        
        if cksize > 0:
            _ = f.read(cksize)
        return 0
        
class lablnote_chunk():
    dwName = ''
    Data = ''
    
    def parse(self,f,cksize):
        self.dwName = readFourcc(f)
        self.Data = f.read(cksize-4)
        print('  dwName:',self.dwName)
        print('  Data:',self.Data)
        return 0



class ltxt_chunk():
    dwName = ''
    dwSampleLength = 0
    dwPurpose = 0
    wCountry = 0
    wLanguage = 0
    wDialect = 0
    wCodePage = 0
    data = ''
    
    def parse(self,f,cksize):
        self.dwName = readFourcc(f)
        self.dwSampleLength = readBytesAsInt(f,4)
        self.dwPurpose = readBytesAsInt(f,4)
        self.wCountry = readBytesAsInt(f,2)
        self.wLanguage = readBytesAsInt(f,2)
        self.wDialect = readBytesAsInt(f,2)
        self.wCodePage = readBytesAsInt(f,2)
        self.data = f.read(cksize-20)
        print('  dwName:',self.dwName)
        print('  data:',self.data)
        return 0


class file_chunk():
    dwName = ''
    dwedtype = 0
    data = ''
    
    def parse(self,f,cksize):
        self.dwName = readFourcc(f)
        self.dwedtype = readBytesAsInt(f,4)
        self.data = bytes(f.read(cksize-8)).hex()
        return 0
        
    
class waveparser(object):
    def __init__(self, wave):
                
        self.fmt = fmt_chunk()
        self.peak = peak_chunk()
        self.fact = fact_chunk()
        self.cue = cue_chunk()
        self.labl = lablnote_chunk()
        self.note = lablnote_chunk()
        self.ltxt = ltxt_chunk()
        self.file = file_chunk()
        
        self.data = []
        if wave is not None:
            print('wave file path :',wave)
            self.parse(open(wave,'rb'))
        
    def parse_adtlchunk(self,f,adtl_size):
        _chunkID = readFourcc(f)
        _cksize = readBytesAsInt(f,4)
        adtl_size -= 8 + _cksize
        print('chunkID:',_chunkID,' cksize:',_cksize)
        if _chunkID == 'labl':
            self.labl.parse(f,_cksize)
        elif _chunkID == 'note':
            self.note.parse(f,_cksize)
        elif _chunkID == 'ltxt':
            self.ltxt.parse(f,_cksize)
        elif _chunkID == 'file':
            self.file.parse(f,_cksize)
        else:
            _ = f.read(_cksize)
        
        return adtl_size
        
    def parse_LISTchunk(self,f,cksize):
        chunkID = readFourcc(f)
        cksize -= 4
        
        print('chunkID:',chunkID)
        
        if chunkID == 'adtl':
            while(cksize>0):
                cksize = self.parse_adtlchunk(f,cksize)
        elif cksize > 0:
            _ = f.read(cksize)
            
        return 0
                
    def parse_subchunk(self,f):
        chunkID = readFourcc(f)
        cksize = readBytesAsInt(f,4)
        print('chunkID:',chunkID, ' cksize:',cksize)
        
        if chunkID == 'fmt ':
            return self.fmt.parse(f,cksize)
        elif chunkID == 'fact':
            return self.fact.parse(f,cksize)
        elif chunkID == 'PEAK':
            return self.peak.parse(f,cksize)
        elif chunkID == 'cue ':
            return self.cue.parse(f,cksize)
        elif chunkID == 'LIST':
            return self.parse_LISTchunk(f,cksize)
        elif chunkID == 'data':
            self.data = f.read(cksize)
            return -1
        
        else:
            _ = bytes(f.read(cksize))
            
        return 0
    
    def parse(self, f):
        if f is None:
            return
        
        chunkID = readFourcc(f)
        cksize = readBytesAsInt(f,4)
        mediaformat = readFourcc(f)
        print('chunkID:',chunkID, ' cksize:',cksize, '  format:', mediaformat)    
        if chunkID != 'RIFF':
            return
        
        while(1):
            ret = self.parse_subchunk(f)
            if ret < 0:
                break    
            
        f.close()
        

if __name__ == "__main__":
    # http://www-mmsp.ece.mcgill.ca/Documents/AudioFormats/WAVE/Samples.html
    wave = waveparser('P1/문제1-5.wav')

wave file path : P1/문제1-5.wav
chunkID: RIFF  cksize: 202314   format: WAVE
chunkID: fmt   cksize: 16
  wFormatTag : 1
  wChannels : 1
  dwSamplePerSec : 16000
  dwAvgBytesPerSec : 32000
  wBlockAlign : 2
  wBitsPerSample : 16
chunkID: data  cksize: 128310
